In [7]:
import pandas as pd

def load_cpi_data(cpi_path):
    df = pd.read_excel(cpi_path)
    df.columns = df.iloc[3][:3].to_list() + df.iloc[1][3:].to_list()
    df = df.iloc[4:]
    df = df.drop(columns=df.columns[df.columns.isna()])
    df = df.melt(id_vars=["Year", "Months"], var_name="inflation_code", value_name="index")
    df = df[(df['inflation_code'].str.len() == 5) | (df['inflation_code'].str.len() == 1)]
    df["inflation_code"] = df["inflation_code"].astype(str)
    return df

def add_weights(df, weights_path):
    weights = pd.read_excel(weights_path, dtype={"inflation_code": str})
    df = df.merge(weights, on=["inflation_code", "Year"], how="left")
    df = df[df.Year > 2018].fillna(100)
    return df

def map_months(df):
    turkish_months = {
        'Ocak': 1, 'Şubat': 2, 'Mart': 3, 'Nisan': 4, 'Mayıs': 5, 'Haziran': 6,
        'Temmuz': 7, 'Ağustos': 8, 'Eylül': 9, 'Ekim': 10, 'Kasım': 11, 'Aralık': 12
    }
    df['Month_Num'] = df['Months'].map(turkish_months)
    df.sort_values(by=['inflation_code', 'Year', 'Month_Num'], inplace=True)
    return df

def merge_prev_december(df):
    dec = df[df['Month_Num'] == 12][['Year', 'inflation_code', 'index']].copy()
    dec['Year'] += 1
    dec.rename(columns={'index': 'Prev_December_Index'}, inplace=True)
    return df.merge(dec, on=['inflation_code', 'Year'], how='left')

def merge_prev_year_weights(df):
    prev_weights = df[['Year', 'Months', 'inflation_code', 'weights']].copy()
    prev_weights['Year'] += 1
    prev_weights.rename(columns={'weights': 'Prev_Year_Weight'}, inplace=True)
    return df.merge(prev_weights, on=['Year', 'Months', 'inflation_code'], how='left')

def merge_prev_year_index(df):
    prev_index = df[['Year', 'Months', 'inflation_code', 'index']].copy()
    prev_index['Year'] += 1
    prev_index.rename(columns={'index': 'Prev_Year_Month_Index'}, inplace=True)
    return df.merge(prev_index, on=['Year', 'Months', 'inflation_code'], how='left')

def merge_general_index(df):
    general = df[df['inflation_code'] == '0'][['Year', 'Months', 'index']].copy()
    general['Year'] += 1
    general.rename(columns={'index': 'Prev_Year_Month_General_Index'}, inplace=True)
    df = df.merge(general, on=['Year', 'Months'], how='left')

    dec_general = df[(df['inflation_code'] == '0') & (df['Month_Num'] == 12)][['Year', 'index']].copy()
    dec_general['Year'] += 1
    dec_general.rename(columns={'index': 'Prev_Year_December_General_Index'}, inplace=True)
    return df.merge(dec_general, on='Year', how='left')

def merge_prev_2_years_december(df):
    general_2y = df[(df['inflation_code'] == '0') & (df['Month_Num'] == 12)][['Year', 'index']].copy()
    general_2y['Year'] += 2
    general_2y.rename(columns={'index': 'Prev_2_Years_December_General_Index'}, inplace=True)
    df = df.merge(general_2y, on='Year', how='left')

    all_codes_2y = df[df['Month_Num'] == 12][['Year', 'inflation_code', 'index']].copy()
    all_codes_2y['Year'] += 2
    all_codes_2y.rename(columns={'index': 'Prev_2_Years_December_Index'}, inplace=True)
    return df.merge(all_codes_2y, on=['Year', 'inflation_code'], how='left')

def calculate_indicators(df):
    df['Cumulative_Inflation_%'] = ((df['index'] / df['Prev_December_Index']) - 1) * 100
    df['last_years_partly_cumulative_Inflation_%'] = (
        (df['Prev_December_Index'] - df['Prev_Year_Month_Index']) / df['Prev_2_Years_December_Index']
    ) * 100

    df["yearly_contribution_part_1"] = df["weights"] * df["Prev_Year_December_General_Index"] / df["Prev_Year_Month_General_Index"]
    df["yearly_contribution_part_2"] = (df["index"] - df["Prev_December_Index"]) / df["Prev_December_Index"]
    df["yearly_contribution_part_3"] = (df["Prev_Year_Weight"] * df["Prev_2_Years_December_General_Index"]) / df["Prev_Year_Month_General_Index"]
    df["yearly_contribution_part_4"] = (df["Prev_December_Index"] - df["Prev_Year_Month_Index"]) / df["Prev_2_Years_December_Index"]

    df["yearly_contribution"] = (
        df["yearly_contribution_part_1"] * df["yearly_contribution_part_2"]
        + df["yearly_contribution_part_3"] * df["yearly_contribution_part_4"]
    )
    return df

def process_cpi_data(cpi_path, weights_path, output_path):
    df = load_cpi_data(cpi_path)
    df = add_weights(df, weights_path)
    df = map_months(df)
    df = merge_prev_december(df)
    df = merge_prev_year_weights(df)
    df = merge_prev_year_index(df)
    df = merge_general_index(df)
    df = merge_prev_2_years_december(df)
    df = calculate_indicators(df)
    df.to_excel(output_path, index=False)
    return df

# === Usage ===
final_df = process_cpi_data(
    "raw_data/harcama gruplarina gore endeks sonuclari.xlsx",
    "raw_data/cpi_item_weights_subgroups_5.xlsx",
    "yearly_contribution.xlsx"
)
final_df.dropna()

/var/folders/lx/dlxz3kpj1pq9fx9tbtzznnr00000gn/T/ipykernel_2432/2999565927.py:16: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df[df.Year > 2018].fillna(100)


,Year,Months,inflation_code,index,weights,Month_Num,Prev_December_Index,Prev_Year_Weight,Prev_Year_Month_Index,Prev_Year_Month_General_Index,Prev_Year_December_General_Index,Prev_2_Years_December_General_Index,Prev_2_Years_December_Index,Cumulative_Inflation_%,last_years_partly_cumulative_Inflation_%,yearly_contribution_part_1,yearly_contribution_part_2,yearly_contribution_part_3,yearly_contribution_part_4,yearly_contribution
24,2021,Ocak,0,513.30,100.000000,1,504.81,100.0000,446.45,446.45,504.81,440.50,440.50,1.681821,13.248581,113.072013,0.016818,98.667264,0.132486,14.973681
25,2021,Şubat,0,517.96,100.000000,2,504.81,100.0000,448.02,448.02,504.81,440.50,440.50,2.604940,12.892168,112.675773,0.026049,98.321504,0.128922,15.610910
26,2021,Mart,0,523.53,100.000000,3,504.81,100.0000,450.58,450.58,504.81,440.50,440.50,3.708326,12.311010,112.035599,0.037083,97.762883,0.123110,16.190244
27,2021,Nisan,0,532.32,100.000000,4,504.81,100.0000,454.43,454.43,504.81,440.50,440.50,5.449575,11.437003,111.086416,0.054496,96.934621,0.114370,17.140154
28,2021,Mayıs,0,537.05,100.000000,5,504.81,100.0000,460.62,460.62,504.81,440.50,440.50,6.386561,10.031782,109.593591,0.063866,95.631974,0.100318,16.592853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2025,Nisan,12700,4755.30,0.509749,4,3626.90,0.4816,3363.52,2207.50,2684.55,1859.38,2286.97,31.111969,11.516548,0.619908,0.311120,0.405652,0.115165,0.239583
11516,2025,Mayıs,12700,4758.60,0.509749,5,3626.90,0.4816,3368.65,2281.85,2684.55,1859.38,2286.97,31.202956,11.292234,0.599709,0.312030,0.392435,0.112922,0.231442
11517,2025,Haziran,12700,4762.11,0.509749,6,3626.90,0.4816,3373.63,2319.29,2684.55,1859.38,2286.97,31.299733,11.074478,0.590028,0.312997,0.386100,0.110745,0.227436
11518,2025,Temmuz,12700,4766.98,0.509749,7,3626.90,0.4816,3383.45,2394.10,2684.55,1859.38,2286.97,31.434007,10.645089,0.571591,0.314340,0.374035,0.106451,0.219490
